In [2]:
from getbibleverse import verse_lookup

import pandas as pd
import numpy as np

from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import sentencepiece

In [3]:
def store_verses(*passage):

    # create empty df to store verses
    verses_df = pd.DataFrame(columns=["bookname", "chapter", "verse", "text"])
    df = pd.DataFrame([])

    # create empty list to store input separately
    books = []
    chapters = []
    verses = []

    # break down input into separate lists
    for entry in passage:
        spaces = 0
        for i in range(0, len(entry)):
            if entry[i] == " ":
                spaces += 1
        
        if spaces == 1:
            book, ch_ver = entry.split(" ")
            chapter, verse_range = ch_ver.split(":")
        elif spaces == 2:
            book_num, book, ch_ver = entry.split(" ")
            book = book_num + " "  + book
            chapter, verse_range = ch_ver.split(":")
        else: 
            ValueError

        books.append(book)
        chapters.append(chapter)
        verses.append(verse_range)

    for i in range(len(books)):
        data = pd.json_normalize(verse_lookup(
            books[i], chapters[i], verses[i]))
        verses_df = pd.concat([verses_df, data], ignore_index=True)

    verses_df['book_chapter'] = verses_df['bookname'] + " " + verses_df['chapter']

    df['book_chapter'] = verses_df["book_chapter"].unique()
    df['verses'] =[i for i in verses]
    df['location'] = df['book_chapter'] + ":" + df['verses']

    verses_df = verses_df.groupby(['bookname', 'chapter'], sort=False).sum().reset_index()

    df['text'] = verses_df['text']

    text = "".join(df['text'])

    return text

In [4]:
text = store_verses("Matthew 20:28", "Colossians 3:12", "Proverbs 22:12")

In [5]:
text

'just as the Son of Man did not come to be served but to serve, and to give his life as a ransom for many.” Therefore, as the elect of God, holy and dearly loved, clothe yourselves with a heart of mercy, kindness, humility, gentleness, and patience, The eyes of the Lord watched over a cause and subverted the words of the treacherous person. '

import os

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import tensorflow as tf
from transformers import T5Tokenizer, T5ForConditionalGeneration


def summarize_text(text):
    # Initialize the tokenizer and model
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    model = T5ForConditionalGeneration.from_pretrained("t5-base")

    # Tokenize the text
    inputs = tokenizer.encode(
        "summarize: " + text, return_tensors="pt", max_length=512, truncation=True
    )

    # Generate a summary
    outputs = model.generate(
        inputs,
        max_length=150,
        min_length=40,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True,
    )

    # Decode the summary
    summary = tokenizer.decode(outputs[0])

    return summary

summarize_text(text)